<a href="https://colab.research.google.com/github/maskot1977/AdvancedTheoryOfPharmacoinformaticsSimulation/blob/KOg4nHg78ZfUM4O2/%E3%83%95%E3%82%A1%E3%83%BC%E3%83%9E%E3%82%B3%E3%82%A4%E3%83%B3%E3%83%95%E3%82%A9%E3%83%9E%E3%83%86%E3%82%A3%E3%82%AF%E3%82%B9%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E7%89%B9%E8%AB%964.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ファーマコインフォマティクスシミュレーション特論
　小寺 正明

# 第４回 ML-based virtual screening

創薬における virtual screening を大別すると Structure-based virtual screening (SBVS) と Ligand-based virtual screening (LBVS) に分かれると思いますが、今回は LBVS を機械学習(ML)に基づいて行う方法を Google Colaboratory 上でデモンストレーションします。今回の講義は以下のような構成で解説します。

- RDKit インストール
- 化合物データ取得
    - 回帰用データを分類用データに変換（練習のため）
- RDKit supporter
    - RDKit 記述子
    - フィンガープリント
- 説明変数
- 多様体学習
- 基本
    - 回帰問題
    - 分類問題
- Ensemble
    - Voting
    - Stacking
    - Bagging
    - Boosting
- クロスバリデーション
- 前処理
    - 特徴選択
- ハイパーパラメーターチューニング
- 最後に

# RDKit インストール

まずは化合物構造を計算機上で取り扱える便利な RDKit をインストールします。

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c rdkit rdkit python=3.7
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

# 化合物データ取得

次に、化合物データを取得しましょう。創薬とは関係がない化合物なのですが、PCCDB というデータベースから少しだけサンプリングした化合物データです。

In [ ]:
import pandas as pd

# csvからのデータ読み込み
url = "https://raw.githubusercontent.com/maskot1977/toydata/main/data/data_18.csv"
df_reg = pd.read_csv(url)
df_reg

この中で、今回は「Melting point」を目的変数としてみましょう。これは次のような分布を持っています。

In [ ]:
Y = df_reg["Melting point"]
Y.hist(bins=20)

## 回帰用データを分類用データに変換（練習のため）

化学情報学（ケモインフォマティクス）では、目的変数を連続値とみなして、回帰問題として解くケースが多くあります。

一方、創薬の世界では、薬理活性の有無などの目的変数を離散値とみなして、分類問題として解くケースのほうが多いです。そして、そのデータには大きな偏りがある場合がほとんどです（少数の分子しか薬理活性を持たない等）。

そこで今回は、次のようにして、目的変数の連続値を偏りを持つ離散値に変換します。

In [ ]:
import numpy as np

df_cla = pd.DataFrame(np.where(df_reg > df_reg.describe().median() * 1.8, 1, 0), columns=df_reg.columns)
df_cla

上記のようにして変換した分類用データでは、目的変数は次のような偏りを持つ離散値になります。

In [ ]:
Y2 = df_cla["Melting point"]
Y2.hist(bins=20)

# RDKit supporter

RDKit supporter は、RDKit や ML 周りで便利な関数やクラスを私が書き溜めたものです。インストールしてみましょう。

In [ ]:
!pip install git+https://github.com/maskot1977/rdkit_supporter.git

## RDKit 記述子

SMILESなどの化学構造情報から算出可能な数字の組のうち、代表的なものが RDKit descriptors です。RDKit supporter を用いて次のように算出できます。

In [ ]:
%%time 
from rdkit_supporter.descriptors import calc_descriptors

rdkit_df = calc_descriptors(df_reg['Open Babel SMILES'])
display(rdkit_df)

## フィンガープリント

RDKit で算出できる数字の組としては他に、各種のフィンガープリントがあります。いろいろあります。

In [ ]:
# rdkit_supporter で取り扱えるフィンガープリントのリスト
from rdkit_supporter.fingerprints import Fingerprinter

fingerprinter = Fingerprinter()
fingerprinter.names

今回は、フィンガープリントの代表として ECFP2 を使ってみましょう。

In [ ]:
fp_type = 'ECFP2'
fp_df = pd.DataFrame([vec for vec in fingerprinter.transform(df_reg['Open Babel SMILES'], fp_type=fp_type)])
fp_df

ここまで来れば、あとは普通の（？）機械学習です。

# 説明変数

今回は、説明変数 X として RDKit descriptors を用います。

In [ ]:
X = rdkit_df

# 多様体学習 (Manifold Learning)

一般に機械学習で用いるデータは高次元なので、データ構造（データの分布等）を可視化するために多様体学習を用います。多様体学習はまた、データを次元縮約して機械学習の説明変数として与えることにより、計算量の削減や、精度の向上に役立つ場合があります。

まずは、線形変換する [主成分分析（PCA）](https://qiita.com/maskot1977/items/082557fcda78c4cdb41f) から。Pythonの機械学習ライブラリとして名高い scikit-learn （略して sklearn） をインポートして用いることができます。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

manifold = PCA()
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y)
plt.show()
print(manifold.explained_variance_ratio_[:10])

おっと、このままでは説明変数に欠損値 (NaN) があるため計算できないようですね。[欠損値を補間するための方法](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute)にはいろいろあります。

*   欠損値を持つ列を丸ごと除去する
*   欠損値をゼロで埋める
*   欠損値を平均値で埋める
*   欠損値を中央値で埋める
*   欠損値を最頻値で埋める
*   欠損値を機械学習で補間する

今回は、そのうち [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) で補間してみましょう。


In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer()
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

すると今度はエラーなく PCA を実行できました。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

manifold = PCA()
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y)
plt.colorbar()
plt.show()
print(manifold.explained_variance_ratio_[:10])

こんな極端な分布が得られるときは、各変数の尺度が大きく異なる時ですね。[StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) などのスケーラーを用いて尺度を揃えましょう。[Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) を用いると、StandardScaler を通してから PCA するというような計算パイプラインが簡単に作れます。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

manifold = Pipeline([('scaler', StandardScaler()), ('pca', PCA())])
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y)
plt.colorbar()
plt.show()
print(manifold['pca'].explained_variance_ratio_[:10])

上記のプロットの色は目的変数の連続値を表しています。 PCA から察するに、この分子の集合はいくつかのサブグループに分かれるように見えますが、それと目的変数との関係は、あるようなないような...？

次は同じPCAプロットを、目的変数の離散値で色付けしてみましょう。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

manifold = Pipeline([('scaler', StandardScaler()), ('pca', PCA())])
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y2)
plt.colorbar()
plt.show()
print(manifold['pca'].explained_variance_ratio_[:10])

うーん、何となくは分かれるみたいですが、何となくですね。

次は、非線形の多様体学習モデルとして有名な [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) を使ってみましょうか。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

manifold = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE())])
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y)
plt.colorbar()
plt.show()

実はこの t-SNE、ハイパーパラメータを変えると構造が大きく変わってしまう、乱数の種が変わると構造が大きく変わってしまう、などの特徴があるため、ひとつの t-SNE マッピングだけしか眺めないというのは危険だったりします。詳しくは [t-SNEの教師ありハイパーパラメーターチューニング](https://qiita.com/maskot1977/items/2213e33c31cfc5403bf6) などをご参照ください。

次は同じt-SNEですが離散値の目的変数で色付けしてみましょう。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

manifold = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE())])
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y2)
plt.colorbar()
plt.show()

うーん、まあ、高い値の化合物がある程度は固まっているようには見えますね。

次は、同じく非線形の多様体学習モデルでありながら、パラメータ依存の少ない [Isomap](https://qiita.com/maskot1977/items/9096a736a7442fd4fbf1) を試してみましょう。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

manifold = Pipeline([('scaler', StandardScaler()), ('isomap', Isomap())])
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y)
plt.colorbar()
plt.show()

離散値の目的変数で色付けしてみましょう。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

manifold = Pipeline([('scaler', StandardScaler()), ('isomap', Isomap())])
embedding = manifold.fit_transform(X)
plt.scatter(embedding[:, 0], embedding[:, 1], alpha=0.5, c=Y2)
plt.colorbar()
plt.show()

さて、とりあえず PCA, t-SNE, Isomap を使ってみましたが、このようにさまざまなマッピングをしてじっくり見比べてみると、化合物空間がどのような構造になっているのか、ぼんやりとしたイメージを持つことができるのではないでしょうか。

他にも私のお気に入りとして [UMAP](https://qiita.com/maskot1977/items/31079c132e3fea3332a5) というものがあるのですが、これは scikit-learn に入っていないので、今回は省略します。

# 基本

## 回帰問題

それでは、機械学習を用いた化合物の物性予測の基本から行きましょう。ハイパーパラメーターチューニングは、しばらく行いません。デフォルトパラメータだけでどこまでできるか、やってみましょう。最初に回帰問題を取り扱います。回帰問題の性能評価指標にはさまざまなものがありますが、詳しくは[こちら](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)をご参照ください。

いくつかの回帰手法を試してみましょう。まずは [SVR (SVM Regressor)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) から。 

In [ ]:
from sklearn.svm import SVR
from rdkit_supporter import depict

model = SVR()
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

全然ダメですね。次は [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)。

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from rdkit_supporter import depict

model = KNeighborsRegressor()
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

次は [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)。

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from rdkit_supporter import depict

model = RandomForestRegressor()
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

次は [GradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)。ちなみにこれの発展型で、大規模データでも速い [HistGradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html) というのがあるそうです。

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from rdkit_supporter import depict

model = GradientBoostingRegressor()
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

最後に [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)。

In [ ]:
from sklearn.neural_network import MLPRegressor
from rdkit_supporter import depict

model = MLPRegressor()
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

## 分類問題

次に、分類問題の基本に取り組んでみましょう。ここでも、ハイパーパラメーターチューニングは行いません。デフォルトパラメータだけでどこまでできるか、やってみましょう。分類問題の性能評価指標にはさまざまなものがありますが、詳しくは[こちら](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)をご参照ください。

いくつかの分類手法を試してみましょう。まずは [SVC (SVM Classifier)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) から。

In [ ]:
from sklearn.svm import SVC
from rdkit_supporter import depict

model = SVC()
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

悪い例を見るのも勉強になりますね（？）。次は [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

次は [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)。

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

次は [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)。ちなみにこれの改良型で、大規模データでも速い [HistGradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) というものがあるそうです。

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

最後に [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)。

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

# Ensemble

ここまで、単独の学習器の性能を見てきました。ここからは、複数の学習器を組み合わせる（ensembleする）とどうなるか見てみたいと思います。

ところで、ensemble は既にこの講義で登場しています。RandomForest や GradientBoosting はそれぞれ `from sklearn.ensemble import RandomForestClassifier` や `from sklearn.ensemble import GradientBoostingClassifier` のようにして import することから分かるように、それ自体が ensemble 手法です。

ensemble 手法を大別すると

- voting (複数の学習器を並列に用いて、平等に取り扱う)
- stacking (複数の学習器を並列に用いて、その結果をさらに学習する)
- bagging (同タイプの学習器を並列に用いて、異なるサンプリングデータで複数回学習する)
- boosting (同タイプの学習器を直列に用いて、前の学習器が間違えた部分を後の学習器が再学習する)

の４種類になります。まずは Voting から行ってみましょう。

## Voting

voting は 「複数の学習器を並列に用いて、平等に取り扱う」 と説明しましたが、その複数の学習器は、同じタイプでも異なるタイプでも構いません。

回帰問題の場合は単純に、（デフォルト設定の場合）複数の学習器の出力の「平均値」を最終的な出力とします（オプションで、各々の学習器に重みづけを設定することが可能です）。詳しくは [VotingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html) 参照。

In [ ]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neural_network import MLPRegressor
from rdkit_supporter import depict

model = VotingRegressor([('svm', SVR()), ('kn', KNeighborsRegressor()), ('mlp', MLPRegressor())])
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

分類問題の場合は（デフォルト設定の場合）複数の学習器の出力の「多数決」を取ります（オプションで、予測された可能性値 predict probability の和が最大のものを選ぶこともできます）。詳しくは [VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) 参照。

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
from rdkit_supporter import depict

model = VotingClassifier([('svm', SVC()), ('kn', KNeighborsClassifier()), ('mlp', MLPClassifier())])
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

## Stacking

Stacking は Voting をさらに発展させたもので、「複数の学習器を並列に用いて、その結果をさらに学習する」 と説明できます。複数の学習器の出力結果を新たな入力として、final_estimator を学習させ最終決定を下します。

回帰問題の場合、デフォルトで [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) が用いられますが、他の回帰モデルを利用することも可能です。詳しくは [StackingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html) 参照。

In [ ]:
sfrom sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from rdkit_supporter import depict
from sklearn.linear_model import RidgeCV

model = StackingRegressor(
    [('svm', SVR()), ('kn', KNeighborsRegressor()), ('mlp', MLPRegressor())],
    final_estimator=RidgeCV())
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

回帰問題の場合、デフォルトで [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) が用いられますが、他の回帰モデルを利用することも可能です。詳しくは [StackingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) 参照。

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from rdkit_supporter import depict
from sklearn.linear_model import LogisticRegression

model = StackingClassifier(
    [('svm', SVC()), ('kn', KNeighborsClassifier()), ('mlp', MLPClassifier())],
    final_estimator=LogisticRegression())
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

## Bagging

Bagging は、「同タイプの学習器を並列に用いて、異なるサンプリングデータで複数回学習する」手法です。[こちら](https://helve-blog.com/posts/python/sklearn-bagging-classifier/) の記事が詳しいです。

まずは [BaggingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) から。

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from rdkit_supporter import depict

model = BaggingRegressor(base_estimator=KNeighborsRegressor())
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

次は [BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html) 。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from rdkit_supporter import depict

model = BaggingClassifier(base_estimator=KNeighborsClassifier())
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)

## Boosting

Boosting は「同タイプの学習器を直列に用いて、前の学習器が間違えた部分を後の学習器が再学習する」と説明できます。まずは [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html) から。

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostRegressor
from rdkit_supporter import depict

model = AdaBoostRegressor(base_estimator=KNeighborsRegressor())
model.fit(X, Y)
depict.regression_metrics(model, X, Y)

次は [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)。

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from rdkit_supporter import depict

model = AdaBoostClassifier(base_estimator=RandomForestClassifier())
model.fit(X, Y2)
depict.classification_metrics(model, X, Y2)


技術的には、今まで述べた手法をいくらでも積み重ねることができます。Ensemble手法の中に別のEnsemble手法を入れたり、さらにそれを Pipeline の中に入れたり、さまざまな複雑化が可能です。

# クロスバリデーション

ここまで、全てのデータを学習に用いていました。しかし本当はこれではいけません。学習用データと検証用データに分割して、交差検証（クロスバリデーション）しないと、汎化性能がある（オーバーフィッティングしていない）かどうかが分かりません。交差検証する方法もさまざまなものがありますが、ここでは [Kfold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) を用いましょう。（交差検証について詳しくは [こちら](https://scikit-learn.org/stable/modules/cross_validation.html)）

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, random_state=53, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(i, train_index, test_index)

この KFold を使って、SVR (SVM Regressor) の交差検証をしてみましょう。rdkit_supporter の depict.kfold_cv を使えば、次のように交差検証が行えます。全データを用いて学習した結果と、交差検証の結果を比較してみましょう。

In [ ]:
from sklearn.svm import SVR
from rdkit_supporter import depict

model = SVR()
models = depict.kfold_cv(model, X, Y)

続いて、次のようにすることで、交差検証で得た学習済み予測器を ensemble した予測モデルを構築できます。ensemble 前の結果と後の結果を比較してみましょう。


In [ ]:
from rdkit_supporter.ensemble import VotingEstimator

ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

同様の工程を、他の予測器に対しても行うことができます。全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

同様に、全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

同様に、全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

同様に、全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor()
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

次は分類モデルです。全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.svm import SVC

model = SVC(probability=True)
models = depict.kfold_cv(model, X, Y2)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.classification_metrics(ensemble, X, Y2)

同様に、全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
models = depict.kfold_cv(model, X, Y2)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.classification_metrics(ensemble, X, Y2)

同様に、全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
models = depict.kfold_cv(model, X, Y2)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.classification_metrics(ensemble, X, Y2)

同様に、全データを用いて学習した結果と、交差検証の結果と、ensembleの結果を比較してみましょう。

In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
models = depict.kfold_cv(model, X, Y2)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.classification_metrics(ensemble, X, Y2)

# 前処理

ここまで、ensemble 手法を用いて複雑なモデルを設計したり、交差検証するところまで行いました。次は、データの前処理がどのように結果に影響を及ぼすか調べてみましょう。

まずは、先ほど計算したものと同じですが、前処理をせずそのまま SVR するとこのようになります。




In [ ]:
from sklearn.svm import SVR

model = SVR()
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

前処理として StandardScaler を用いた Pipeline を作るとどのようになるでしょうか。

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

model = Pipeline([('scaler', StandardScaler()), ('svm', SVR())])
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

StandardScaler を通してから PCA を通したものを SVR するとどうなるでしょうか。

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

model = Pipeline([('scaler', StandardScaler()), ('pca', PCA()), ('svm', SVR())])
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

PCA の長所の一つは、第一主成分から順番に、分散の大きい成分をピックアップできることです。それらが精度に与える影響を調べてみましょう。

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

for n_components in [2, 10, 50, 100, 150]:
    print("n_components=", n_components)
    model = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=n_components)), ('svm', SVR())])
    models = depict.kfold_cv(model, X, Y)
    ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
    depict.regression_metrics(ensemble, X, Y)

## 特徴選択

これも前処理の一つと言えますが、回帰や分類に対する寄与の大きい特徴量だけを選んで学習させると、精度が向上することがあります。特徴の重要度の算出方法は多数ありますが、そのうちの一つが RandomForest によるものです。

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from rdkit_supporter import depict

model = RandomForestClassifier()
model.fit(X, Y2)
depict.feature_importances(model, X)

興味深いことに、同じデータに対して同じ事象を予測しようとしても、回帰問題として考えた時と、分類問題として考えた時で、予測に寄与する特徴量は異なってしまうことが多いです。（ですので、その解釈には気をつけなければなりません。）

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from rdkit_supporter import depict

model = RandomForestRegressor()
model.fit(X, Y)
depict.feature_importances(model, X)

GradientBoosting も同様に、特徴量の重要度を算出できます。

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from rdkit_supporter import depict

model = GradientBoostingRegressor()
model.fit(X, Y)
depict.feature_importances(model, X)

それでは、上記のようにして得られた重要な特徴量のトップいくつかをピックアップして、それのみを説明変数とした予測器の性能の変化を追ってみましょう。

In [ ]:
selector = model

In [ ]:
from rdkit_supporter.preprocess import FeatureMasker, features_top
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

for top in [2, 10, 50, 100, 150]:
    print("top=", top)
    model = Pipeline(
        [
            ("fm", FeatureMasker(
                remaining_cols=features_top(top, selector, X)
                )
            ),
            ("scaler", StandardScaler()),
            ("svm", SVR()),
        ]
    )
    models = depict.kfold_cv(model, X, Y)
    ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
    depict.regression_metrics(ensemble, X, Y)

# ハイパーパラメーターチューニング

第４回講義の最後に、ようやくハイパーパラメーターチューニングを行います。ここまで、ハイパーパラメーターチューニングは行いませんでした。ハイパーパラメーターチューニングに頼らなくても、その他の工夫で予測性がけっこう向上できるということがお分かりいただけたのではないかと思います。ハイパーパラメーターチューニングは普通、時間がかかります。ですので、ハイパーパラメーターチューニングに頼るのは、最後の方で良いのかもしれません。ハイパーパラメーターチューニングについて詳しくは第１回講義で解説しましたので、復習してみてください。

In [ ]:
from sklearn.model_selection import GridSearchCV
top = 50
parameters = dict({ 
    "svm__C": [1, 10, 100], 
              #"svm__kernel":["poly", "rbf", "linear"],
              #"svm__gamma": ["auto", "scale"], 
              #"svm__degree": [1, 3, 5],
              #"svm__max_iter":[530000]
              }
              )
model = GridSearchCV(
    Pipeline(
        [
            ("fm", FeatureMasker(
                remaining_cols=features_top(top, selector, X)
                )
            ),
            ("scaler", StandardScaler()),
            ("svm", SVR()),
        ]
    ), parameters)
models = depict.kfold_cv(model, X, Y)
ensemble = VotingEstimator([(str(n), m) for n, m in enumerate(models)])
depict.regression_metrics(ensemble, X, Y)

# 課題

1. この講義では、RDKit descriptor を説明変数とした予測を行いました。ECFP2 フィンガープリントを説明変数として用いて同じ計算を行い、得られた結果を RDKit descriptor による結果と比較してください。

2. RDKit descriptor を説明変数とした予測と、ECFP2 フィンガープリントを説明変数とした予測の両方を有効活用した予測を行うにはどうすればよいか考察してください。

3. ここまで、機械学習的な側面に注目してきましたが、予測結果が得られれば、それについてドメイン（その分野）特有の解釈を入れる必要があります。今回用いた分子は創薬とは関係のない分子でしたが、もしこのような機械学習手法を創薬に活用するならば、どのような分子に対して、どのような目的変数を考え、予測結果に対してどのような解釈を加えると良いのか、例を挙げて説明してください。

## 提出方法：

下記のいずれかの方法で提出してください。

- Google Colaboratory 上で動作させたコードを ikemenmaskot@gmail.com に「共有」

- Jupyter Notebook 上で動作させたコードを ipynb 形式または html 形式にして ikemenmaskot@gmail.com に「メール送信」